# Data Integrity Check 

This notebook validates that the final `feature_engineered` table does not contain
invalid raw flow records.

## Goals:
1. Identify and remove rows where any canonical raw field is NULL, Negative or 0 (which should not occur in valid flow data)

In [1]:
!pip -q install "PyAthena[SQLAlchemy]" sqlalchemy s3fs

In [2]:
import boto3
import sagemaker
import pandas as pd
import numpy as np
from sqlalchemy import create_engine, text

# Display settings
pd.set_option("display.max_rows", 100)
pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", None)
pd.set_option("display.width", None)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


## Connect to Athena

In [3]:
sess = sagemaker.Session()
region = boto3.Session().region_name

results_bucket = sess.default_bucket()
athena_results_path = f"s3://{results_bucket}/athena/staging/"

database_name = "aai540_eda"

engine = create_engine(
    f"awsathena+rest://@athena.{region}.amazonaws.com:443/{database_name}",
    connect_args={"s3_staging_dir": athena_results_path, "region_name": region},
)
print("Region:", region)
print("Athena results:", athena_results_path)

Region: us-east-1
Athena results: s3://sagemaker-us-east-1-128131109986/athena/staging/


In [4]:
# Helper functions for queries
def exec_ddl(sql: str):
    with engine.begin() as conn:
        conn.execute(text(sql))

def read_sql(sql: str) -> pd.DataFrame:
    return pd.read_sql(sql, engine)

## Profile counts: NULL / negative / zero (per feature)

This creates a new table version: `merged_canonical_normalized_v1`

In [5]:
# drop table if it already exists
exec_ddl(f"DROP TABLE IF EXISTS {database_name}.merged_canonical_normalized_v1")

# create table pkt_rate
exec_ddl(f"""
CREATE TABLE {database_name}.merged_canonical_normalized_v1
WITH (
  format = 'PARQUET',
  external_location = 's3://{results_bucket}/aai540/processed/merged_canonical_normalized_v1/',
  parquet_compression = 'SNAPPY'
) AS
SELECT
  *,
  CASE
    WHEN duration IS NULL OR duration <= 0 THEN NULL
    ELSE CAST(pkt_total AS DOUBLE) / CAST(duration AS DOUBLE)
  END AS pkt_rate
FROM {database_name}.merged_canonical_normalized
""")

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /opt/conda/lib/python3.12/site-packages/sqlalchemy/engine/base.py:1967 in _exec_single_context   │
│                                                                                                  │
│   1964 │   │   │   │   │   │   │   evt_handled = True                                            │
│   1965 │   │   │   │   │   │   │   break                                                         │
│   1966 │   │   │   │   if not evt_handled:                                                       │
│ ❱ 1967 │   │   │   │   │   self.dialect.do_execute(                                              │
│   1968 │   │   │   │   │   │   cursor, str_statement, effective_parameters, context              │
│   1969 │   │   │   │   │   )                                                                     │
│   1970                                                                                           │
│                                                                                                  │
│ /opt/conda/lib/python3.12/site-packages/pyathena/sqlalchemy/base.py:391 in do_execute            │
│                                                                                                  │
│   388 │   │   if on_start_query_execution is not None:                                           │
│   389 │   │   │   cursor.execute(statement, parameters, on_start_query_execution=on_start_quer   │
│   390 │   │   else:                                                                              │
│ ❱ 391 │   │   │   cursor.execute(statement, parameters)                                          │
│   392 │                                                                                          │
│   393 │   def do_rollback(self, dbapi_connection: "PoolProxiedConnection") -> None:              │
│   394 │   │   # No transactions for Athena                                                       │
│                                                                                                  │
│ /opt/conda/lib/python3.12/site-packages/pyathena/cursor.py:219 in execute                        │
│                                                                                                  │
│   216 │   │   │   │   self._retry_config,                                                        │
│   217 │   │   │   )                                                                              │
│   218 │   │   else:                                                                              │
│ ❱ 219 │   │   │   raise OperationalError(query_execution.state_change_reason)                    │
│   220 │   │   return self                                                                        │
│   221 │                                                                                          │
│   222 │   def executemany(                                                                       │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
OperationalError: HIVE_PATH_ALREADY_EXISTS: Target directory for table 'aai540_eda.merged_canonical_normalized_v1' 
already exists: s3://sagemaker-us-east-1-128131109986/aai540/processed/merged_canonical_normalized_v1/. You may 
need to manually clean the data at location 
's3://sagemaker-us-east-1-128131109986/athena/staging/tables/b755b2c9-3ade-4483-9a6e-65fb957a1c59' before retrying.
Athena will not delete data in your account.

The above exception was the direct cause of the following exception:

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:5                                                                                    │
│                                                                                                  │
│    2 exec_ddl(f"DROP TABLE IF EXISTS {database_name}.merge

In [6]:
read_sql(f"""
SELECT duration, pkt_total, pkt_rate
FROM {database_name}.merged_canonical_normalized_v1
pkt_total
Limit 25
""")

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /opt/conda/lib/python3.12/site-packages/sqlalchemy/engine/base.py:1967 in _exec_single_context   │
│                                                                                                  │
│   1964 │   │   │   │   │   │   │   evt_handled = True                                            │
│   1965 │   │   │   │   │   │   │   break                                                         │
│   1966 │   │   │   │   if not evt_handled:                                                       │
│ ❱ 1967 │   │   │   │   │   self.dialect.do_execute(                                              │
│   1968 │   │   │   │   │   │   cursor, str_statement, effective_parameters, context              │
│   1969 │   │   │   │   │   )                                                                     │
│   1970                                                                                           │
│                                                                                                  │
│ /opt/conda/lib/python3.12/site-packages/pyathena/sqlalchemy/base.py:391 in do_execute            │
│                                                                                                  │
│   388 │   │   if on_start_query_execution is not None:                                           │
│   389 │   │   │   cursor.execute(statement, parameters, on_start_query_execution=on_start_quer   │
│   390 │   │   else:                                                                              │
│ ❱ 391 │   │   │   cursor.execute(statement, parameters)                                          │
│   392 │                                                                                          │
│   393 │   def do_rollback(self, dbapi_connection: "PoolProxiedConnection") -> None:              │
│   394 │   │   # No transactions for Athena                                                       │
│                                                                                                  │
│ /opt/conda/lib/python3.12/site-packages/pyathena/cursor.py:219 in execute                        │
│                                                                                                  │
│   216 │   │   │   │   self._retry_config,                                                        │
│   217 │   │   │   )                                                                              │
│   218 │   │   else:                                                                              │
│ ❱ 219 │   │   │   raise OperationalError(query_execution.state_change_reason)                    │
│   220 │   │   return self                                                                        │
│   221 │                                                                                          │
│   222 │   def executemany(                                                                       │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
OperationalError: TABLE_NOT_FOUND: line 2:6: Table 'awsdatacatalog.aai540_eda.merged_canonical_normalized_v1' does 
not exist

The above exception was the direct cause of the following exception:

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 read_sql(f"""                                                                                │
│   2 SELECT duration, pkt_total, pkt_rate                                                         │
│   3 FROM {database_name}.merged_canonical_normalized_v1                                          │
│   4 pkt_total                                                   

### Sanity check

In [ ]:
read_sql(f"""
SELECT
  COUNT(*) AS rows_total,
  SUM(CASE WHEN duration IS NULL OR duration <= 0 THEN 1 ELSE 0 END) AS bad_duration_rows,
  SUM(CASE WHEN (duration IS NULL OR duration <= 0) AND pkt_rate IS NOT NULL THEN 1 ELSE 0 END) AS pkt_rate_should_be_null_but_isnt
FROM {database_name}.merged_canonical_normalized_v1
""")


This creates a new table version: `merged_canonical_normalized_v3`

In [ ]:
# drop v3 if it already exists
exec_ddl(f"DROP TABLE IF EXISTS {database_name}.merged_canonical_normalized_v3")

# create v3 with bytes_per_pkt (built on v2 so prior features are retained)
exec_ddl(f"""
CREATE TABLE {database_name}.merged_canonical_normalized_v3
WITH (
  format = 'PARQUET',
  external_location = 's3://{results_bucket}/aai540/processed/merged_canonical_normalized_v3/',
  parquet_compression = 'SNAPPY'
) AS
SELECT
  *,
  CASE
    WHEN pkt_total IS NULL OR pkt_total <= 0 THEN NULL
    ELSE CAST(bytes_total AS DOUBLE) / CAST(pkt_total AS DOUBLE)
  END AS bytes_per_pkt
FROM {database_name}.merged_canonical_normalized_v2
""")

In [ ]:
read_sql(f"""
SELECT duration, pkt_total, bytes_total, bytes_per_pkt
FROM {database_name}.merged_canonical_normalized_v3
WHERE bytes_total > 0
LIMIT 25
""")

### Sanity check

In [ ]:
read_sql(f"""
SELECT
  COUNT(*) AS rows_total,
  SUM(CASE WHEN pkt_total IS NULL OR pkt_total <= 0 THEN 1 ELSE 0 END) AS bad_pkt_total_rows,
  SUM(CASE WHEN (pkt_total IS NULL OR pkt_total <= 0) AND bytes_per_pkt IS NOT NULL THEN 1 ELSE 0 END) AS bytes_per_pkt_should_be_null_but_isnt
FROM {database_name}.merged_canonical_normalized_v3
""")

This creates a new table version: `merged_canonical_normalized_v4`

In [ ]:
# drop v4 if it already exists
exec_ddl(f"DROP TABLE IF EXISTS {database_name}.merged_canonical_normalized_v4")

# create v4 with pkt_ratio (built on v3 so prior features are retained)
exec_ddl(f"""
CREATE TABLE {database_name}.merged_canonical_normalized_v4
WITH (
  format = 'PARQUET',
  external_location = 's3://{results_bucket}/aai540/processed/merged_canonical_normalized_v4/',
  parquet_compression = 'SNAPPY'
) AS
SELECT
  *,
  CASE
    WHEN pkt_fwd IS NULL OR pkt_fwd < 0 THEN NULL
    WHEN pkt_bwd IS NULL OR pkt_bwd < 0 THEN NULL
    ELSE CAST(pkt_fwd AS DOUBLE) / (CAST(pkt_bwd AS DOUBLE) + 1.0)
  END AS pkt_ratio
FROM {database_name}.merged_canonical_normalized_v3
""")

In [ ]:
read_sql(f"""
SELECT duration, pkt_total, pkt_fwd, pkt_bwd, pkt_ratio
FROM {database_name}.merged_canonical_normalized_v4
LIMIT 25
""")

This creates a new table version: `merged_canonical_normalized_v5`

In [ ]:
# drop v5 if it already exists
exec_ddl(f"DROP TABLE IF EXISTS {database_name}.merged_canonical_normalized_v5")

# Create v5 with byte_ratio (built on v4 so prior features are retained)
exec_ddl(f"""
CREATE TABLE {database_name}.merged_canonical_normalized_v5
WITH (
  format = 'PARQUET',
  external_location = 's3://{results_bucket}/aai540/processed/merged_canonical_normalized_v5/',
  parquet_compression = 'SNAPPY'
) AS
SELECT
  *,
  CASE
    WHEN bytes_fwd IS NULL OR bytes_fwd < 0 THEN NULL
    WHEN bytes_bwd IS NULL OR bytes_bwd < 0 THEN NULL
    ELSE CAST(bytes_fwd AS DOUBLE) / (CAST(bytes_bwd AS DOUBLE) + 1.0)
  END AS byte_ratio
FROM {database_name}.merged_canonical_normalized_v4
""")

In [ ]:
read_sql(f"""
SELECT
  bytes_fwd,
  bytes_bwd,
  byte_ratio
FROM {database_name}.merged_canonical_normalized_v5
LIMIT 25
""")

## Finalize Feature Engineering Table

In [ ]:
# drop final table if it already exists
exec_ddl(f"DROP TABLE IF EXISTS {database_name}.feature_engineered")

# create final feature_engineered table from v5
exec_ddl(f"""
CREATE TABLE {database_name}.feature_engineered
WITH (
  format = 'PARQUET',
  external_location = 's3://{results_bucket}/aai540/processed/feature_engineered/',
  parquet_compression = 'SNAPPY'
) AS
SELECT *
FROM {database_name}.merged_canonical_normalized_v5
""")


In [ ]:
read_sql(f"""
SELECT
  pkt_rate,
  byte_rate,
  bytes_per_pkt,
  pkt_ratio,
  byte_ratio
FROM {database_name}.feature_engineered
LIMIT 10
""")

In [ ]:
profile_invalid = read_sql(f"""
SELECT
  COUNT(*) AS total_rows,

  -- NULL counts
  SUM(CASE WHEN duration  IS NULL THEN 1 ELSE 0 END) AS duration_null,
  SUM(CASE WHEN pkt_total IS NULL THEN 1 ELSE 0 END) AS pkt_total_null,
  SUM(CASE WHEN bytes_total IS NULL THEN 1 ELSE 0 END) AS bytes_total_null,
  SUM(CASE WHEN pkt_fwd   IS NULL THEN 1 ELSE 0 END) AS pkt_fwd_null,
  SUM(CASE WHEN pkt_bwd   IS NULL THEN 1 ELSE 0 END) AS pkt_bwd_null,
  SUM(CASE WHEN bytes_fwd IS NULL THEN 1 ELSE 0 END) AS bytes_fwd_null,
  SUM(CASE WHEN bytes_bwd IS NULL THEN 1 ELSE 0 END) AS bytes_bwd_null,

  -- Negative counts
  SUM(CASE WHEN duration  < 0 THEN 1 ELSE 0 END) AS duration_neg,
  SUM(CASE WHEN pkt_total < 0 THEN 1 ELSE 0 END) AS pkt_total_neg,
  SUM(CASE WHEN bytes_total < 0 THEN 1 ELSE 0 END) AS bytes_total_neg,
  SUM(CASE WHEN pkt_fwd   < 0 THEN 1 ELSE 0 END) AS pkt_fwd_neg,
  SUM(CASE WHEN pkt_bwd   < 0 THEN 1 ELSE 0 END) AS pkt_bwd_neg,
  SUM(CASE WHEN bytes_fwd < 0 THEN 1 ELSE 0 END) AS bytes_fwd_neg,
  SUM(CASE WHEN bytes_bwd < 0 THEN 1 ELSE 0 END) AS bytes_bwd_neg,

  -- Zero counts
  SUM(CASE WHEN duration  = 0 THEN 1 ELSE 0 END) AS duration_zero,
  SUM(CASE WHEN pkt_total = 0 THEN 1 ELSE 0 END) AS pkt_total_zero,
  SUM(CASE WHEN bytes_total = 0 THEN 1 ELSE 0 END) AS bytes_total_zero

FROM {database_name}.feature_engineered
""")
profile_invalid

## Count rows to be dropped

In [ ]:
invalid_row_summary = read_sql(f"""
SELECT
  COUNT(*) AS total_rows,
  SUM(
    CASE WHEN
      -- NULL raw features
      duration IS NULL OR pkt_total IS NULL OR bytes_total IS NULL OR
      pkt_fwd IS NULL OR pkt_bwd IS NULL OR bytes_fwd IS NULL OR bytes_bwd IS NULL OR

      -- negative raw features
      duration < 0 OR pkt_total < 0 OR bytes_total < 0 OR
      pkt_fwd < 0 OR pkt_bwd < 0 OR bytes_fwd < 0 OR bytes_bwd < 0 OR

      -- zeros 
      duration = 0 OR pkt_total = 0
    THEN 1 ELSE 0 END
  ) AS rows_to_drop
FROM {database_name}.feature_engineered
""")

invalid_row_summary


## Inspect a few invalid rows

In [ ]:
invalid_rows_sample = read_sql(f"""
SELECT
  duration, pkt_total, bytes_total, pkt_fwd, pkt_bwd, bytes_fwd, bytes_bwd,
  pkt_rate, byte_rate, bytes_per_pkt, pkt_ratio, byte_ratio,
  label, source_dataset
FROM {database_name}.feature_engineered
WHERE
  duration IS NULL OR pkt_total IS NULL OR bytes_total IS NULL OR
  pkt_fwd IS NULL OR pkt_bwd IS NULL OR bytes_fwd IS NULL OR bytes_bwd IS NULL OR
  duration < 0 OR pkt_total < 0 OR bytes_total < 0 OR
  pkt_fwd < 0 OR pkt_bwd < 0 OR bytes_fwd < 0 OR bytes_bwd < 0 OR
  duration = 0 OR pkt_total = 0
LIMIT 20
""")

invalid_rows_sample


## Create cleaned table (drop ALL invalid rows: NULL, negative, or zero)

In [ ]:
# create a cleaned table
exec_ddl(f"DROP TABLE IF EXISTS {database_name}.feature_engineered_cleaned")

exec_ddl(f"""
CREATE TABLE {database_name}.feature_engineered_cleaned
WITH (
  format = 'PARQUET',
  external_location = 's3://{results_bucket}/aai540/processed/feature_engineered_cleaned/',
  parquet_compression = 'SNAPPY'
) AS
SELECT *
FROM {database_name}.feature_engineered
WHERE
  -- Raw canonical must exist
  duration IS NOT NULL
  AND pkt_total IS NOT NULL
  AND bytes_total IS NOT NULL
  AND pkt_fwd IS NOT NULL
  AND pkt_bwd IS NOT NULL
  AND bytes_fwd IS NOT NULL
  AND bytes_bwd IS NOT NULL

  -- Raw canonical must be non-negative
  AND duration >= 0
  AND pkt_total >= 0
  AND bytes_total >= 0
  AND pkt_fwd >= 0
  AND pkt_bwd >= 0
  AND bytes_fwd >= 0
  AND bytes_bwd >= 0

  -- Drop zero denominators
  AND duration > 0
  AND pkt_total > 0
""")

## Validate

In [ ]:
read_sql(f"""
SELECT
  (SELECT COUNT(*) FROM {database_name}.feature_engineered) AS before_rows,
  (SELECT COUNT(*) FROM {database_name}.feature_engineered_cleaned) AS after_rows
""")

In [ ]:
label_distribution = read_sql(f"""
SELECT
  label,
  COUNT(*) AS row_count,
  COUNT(*) * 100.0 / SUM(COUNT(*)) OVER () AS percentage
FROM {database_name}.feature_engineered_cleaned
GROUP BY label
ORDER BY label
""")

label_distribution

In [ ]:
attack_type_distribution = read_sql(f"""
SELECT
  attack_category,
  COUNT(*) AS row_count,
  COUNT(*) * 100.0 / SUM(COUNT(*)) OVER () AS percentage
FROM {database_name}.feature_engineered_cleaned
GROUP BY attack_category
ORDER BY row_count DESC
""")

attack_type_distribution